In [ ]:
import os, os.path, datetime
import keras
from google.colab import drive 
drive.mount('/content/drive') 

# import PredictPower_functions
%cd /content/drive/My\ Drive/Running Plantiga Project/Code
import functions_classification as fc
import functions_models as fm
#import PredictPower_functions as ppf
#import openloop_algorithms as oa


#check if GPU is available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))


%cd /content/drive/My\ Drive/Running Plantiga Project
dir_root = os.getcwd() + '/'

"""some variables"""

if 1:    
    """just for now, delete this again later and put in run_classification"""
    data_name = "5k_no_spectrogram"
    subjects = [11,12,13,15,16,17,18,19,20]
    val_split = 0.2
    test_split = 0.2
    flag_shuffle_train = 0
    flag_plot = 0
    flag_speed_classification = 1
    epochs = 5
    dropout = 0.2
    batch_size = 32
    final_layer_size = 4
    early_stopping_min_delta = 0
    early_stopping_patience = 5
    input_layer_size = 2

    
    """directories"""
    dir_data = dir_root + 'Data/'
    dir_prepared = dir_data + 'Prepared/'
    dir_tfr = dir_prepared + "tfrecords/" + data_name + '/'
    dir_tfr_treadmill = dir_tfr + 'Treadmill/'
    dir_tfr_overground = dir_tfr + 'Overground/'
    dir_results_weights = dir_data + 'Results/' + 'model_weights/'
    dir_results_history = dir_data + 'Results/' + 'model_history/'
    
    
    if flag_speed_classification:
        train_filenames, val_filenames, test_filenames = fc.get_filenames_speed_classification(subjects, test_split, val_split, dir_tfr_treadmill, flag_shuffle_train)
    
    """
    #to doublecheck
    (tf.data.TFRecordDataset(train_filenames)
     .map(fc.parse_tfrecord_rnn)
     .map(fc.prepare_sample)
    )
    """
    
    
    # plot some data
    if flag_plot:
        tense = list()
        for batch in tf.data.TFRecordDataset(train_filenames).map(fc.parse_tfrecord_rnn):
            #print(batch)
            tense.append(batch)
            #break
            
        fc.plot_accelerations(tense)
        
                 
    
    """fun part"""
    final_activation = 'sigmoid'
    if final_layer_size > 1:
        final_activation = 'softmax'
        
    
    if 0:
        model = fm.parallel_channels_conv1D(final_layer_size, final_activation, dropout)
        
    if 1: 
       model = fm.parallel_channels_LSTM(final_layer_size, final_activation, dropout)
    
    
    loss_to_use = tf.keras.losses.BinaryCrossentropy()
    if final_layer_size > 1:
        loss_to_use = 'categorical_crossentropy'
    
    model.compile(loss = loss_to_use, 
                  optimizer = "adam", 
                  metrics = ["accuracy", 
                             #tf.keras.metrics.AUC(curve = 'ROC'),
                             #tf.keras.metrics.AUC(curve = 'PR'),
                             #tf.keras.metrics.Precision(),
                             #tf.keras.metrics.Recall(),
                             #tf.keras.metrics.PrecisionAtRecall(0.8) 
                            ]) #tf.keras.metrics.AUC(from_logits=True)
    
    if input_layer_size > 1:
        train_dataset = fc.get_dataset_multipleinputs(train_filenames, batch_size)
        val_dataset = fc.get_dataset_multipleinputs(val_filenames, batch_size)
    else:
        train_dataset = fc.get_dataset_singleinput(train_filenames, batch_size)
        val_dataset = fc.get_dataset_singleinput(val_filenames, batch_size)
    
    
    "callbacks"
    """
    print('model checkpoint included')
    check_point = keras.callbacks.ModelCheckpoint(filepath= dir_results_weights + model_name + '.h5',
                                                 verbose = 1,
                                                 monitor="val_loss",
                                                 save_best_only=True,
                                                 mode="min", # if we save_best_only, we need to specify on what rule. Rule here is if val_loss is minimum, it owerwrites
                                                 save_weights_only = True,  # to only save weights, otherwise it will save whole model
                                                 )
    
    
    """
    # make sure to add this to the fit model again when uncommenting
    print('early stopping included') 
    earlystopping = tf.keras.callbacks.EarlyStopping( 
                    monitor='val_loss',
                    min_delta=early_stopping_min_delta,
                    patience=early_stopping_patience,
                    verbose=1,
                    mode='auto',
                    restore_best_weights=True # Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.
                    )
    
    #steps_per_epoch = steps_per_epoch
    
    model.fit(train_dataset,
              validation_data = val_dataset, 
              callbacks=[earlystopping],
              #steps_per_epoch = steps_per_epoch,
              #validation_steps = validation_steps, 
              epochs = epochs
             )
    
    
    
    if flag_speed_classification:
        fc.performance_test_dataset(model, test_filenames, batch_size, final_layer_size)
    

Mounted at /content/drive
/content/drive/My Drive/Running Plantiga Project/Code


Found GPU at: /device:GPU:0
/content/drive/My Drive/Running Plantiga Project
Train: 21
Validation: 3
Test: 3


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_left_acc_x (InputLayer)  [(None, 5000, 1)]    0           []                               
                                                                                                  
 input_left_acc_y (InputLayer)  [(None, 5000, 1)]    0           []                               
                                                                                                  
 LSTM1_left_acc_x (LSTM)        (None, 5000, 64)     16896       ['input_left_acc_x[0][0]']       
                                                                                                  
 LSTM1_left_acc_y (LSTM)        (None, 5000, 64)     16896       ['input_left_acc_y[0][0]']       
                                                                                              

KeyboardInterrupt: ignored